In [2]:
import logging
import os

import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist
from pyro.infer import SVI
import torch.distributions.constraints as constraints

/var/folders/ll/n97j7j6n4hl40jzcp14wv24m0000gn/T/ipykernel_15224/4051078519.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/lollipop/miniconda3/envs/pyro/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.9.0')
pyro.enable_validation(True)
pyro.set_rng_seed(1)
logging.basicConfig(format='%(message)s', level=logging.INFO)

We're modeling a joint distribution over observation matrix $D$, predicability matrix $R$, truth matrix $T$, and latent types $Z$.

$D$ is an IxJ matrix where $I$ is the number of objects and $J$ is the number of predicates. $R$ is a $\tau$xJ matrix, where $\tau$ is the number of latent types. $T$ is a OxJ matrix. $Z$ is a 1x$\tau$ matrix. All matrices are boolean, with exception to $Z$.

$D_{i,j}$ refers to the observation of object $i$ being predicated by function $j$. $R_{i,j}$ refers to whether an object $i$ can be predicated by $j$, with $0$ meaning ``no`` and $1$ meaning ``yes``. $T_{i.j}$ refers to the expected truth-value of applying predicate $j$ to object $i$. $Z_i$ refers to the type of object $i$.

$$
    p(D, R, T, Z) \propto p(D | R, T, Z)p(T | R, Z)p(R|Z)p(Z)
$$

In [8]:
def create_data_object_types(alpha=2,num_objs=10):
  '''Function for generating cluster assignments from a CRP.

  Arguments:
    alpha: Concentration parameter.
    n_objects: Number of objects to be clustered.

  Returns:
    cluster_assignments: Cluster assignments for each object.
  '''
  cluster_frequencies = [1]
  cluster_assignments =[0]
  n = num_objs-1
  probs = torch.tensor([1/(1+alpha),alpha/(1+alpha)])
  for i in range(n):
    z_i = dist.Categorical(probs)().item()
    total_customer = i + 1
    if z_i in cluster_assignments:
      cluster_frequencies[z_i] += 1
    else:
      cluster_frequencies += [ 1]
    probs = [n/(total_customer + alpha) for n in cluster_frequencies+[alpha]]
    probs = torch.tensor(probs)
    cluster_assignments.append(z_i)
  return torch.tensor(cluster_assignments)

In [9]:
def create_data_pred_matrix(obj_types, num_preds, theta):
  '''Function for generating predicability matrices.

  Arguments:
    obj_types: Tensor of object type assignments.
    num_preds: Number of predicates.
    theta: Sucess parameter for Bernoulli.

  Returns:
    pred_matrix: Predicability matrix.
  '''
  num_types = len(torch.unique(obj_types))
  pred_matrix = torch.bernoulli(torch.ones(num_types, num_preds)*theta)
  return pred_matrix

In [10]:
def create_data_truth_matrix(pred_matrix, obj_types, pi):
  '''Function for generating truth-value matrices.

  Arguments:
    pred_matrix: Predicability matrix.
    obj_types: List of object type assignments.
    pi: Success parameter for Bernoulli.

  Returns:
    truth_matrix: Predicability matrix.
  '''
  num_objs = len(obj_types)
  num_preds = pred_matrix.size()[1]
  truth_matrix = torch.zeros(num_objs, num_preds)
  for i in range(num_objs):
    for j in range(num_preds):
      if pred_matrix[z[i],j]==1:
        truth_matrix[i,j] = pyro.sample("t", dist.Bernoulli(pi)).item()
      else:
        truth_matrix[i,j] = 0
  return truth_matrix

In [11]:
def create_data_freq_matrix(truth_matrix, pred_matrix, obj_types):
  '''Function for generating frequency matrices.

  Arguments:
    pred_matrix: Predicability matrix.
    obj_types: List of object type assignments.
    truth_matrix: Truth matrix.

  Returns:
    freq_matrix: Frequency matrix.
  '''
  num_objs = len(obj_types)
  num_preds = pred_matrix.size()[1]
  freq_matrix = torch.zeros(num_objs, num_preds)
  for i in range(num_objs):
    for j in range(num_preds):
      if pred_matrix[z[i],j]==1:
        freq_matrix[i,j] = truth_matrix[i,j]
      else:
        freq_matrix[i,j] = -1
  return freq_matrix

# Synthetic data generation

In [12]:
# Number of objects
num_objs = 5
# Number of predicates
num_preds = 5
# Concentration param
alpha = 10
# Success parameter for predicability
theta = pyro.sample("theta", dist.Beta(1,1)).item()
# Success parameter for truth
pi = pyro.sample("pi", dist.Beta(1,1)).item()
print(f"Theta {theta} Pi {pi}")

Theta 0.600306510925293 Pi 0.4010412395000458


In [13]:
# Object type assignments
z = create_data_object_types(alpha=alpha, num_objs=num_objs)
print("Object type assignments:")
print(z)

Object type assignments:
tensor([0, 1, 2, 1, 0])


In [14]:
r = create_data_pred_matrix(obj_types=z, num_preds=num_preds, theta=theta)
print("Predicability matrix:")
r

Predicability matrix:


tensor([[0., 1., 1., 0., 1.],
        [0., 1., 1., 1., 1.],
        [0., 1., 0., 0., 1.]])

In [15]:
t = create_data_truth_matrix(pred_matrix=r, obj_types=z, pi=pi)
print("Truth matrix:")
t

Truth matrix:


tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1.],
        [0., 0., 1., 0., 0.]])

In [16]:
d = create_data_freq_matrix(truth_matrix=t, pred_matrix=r, obj_types=z)
print("Freq matrix:")
d

Freq matrix:


tensor([[-1.,  1.,  0., -1.,  0.],
        [-1.,  1.,  0.,  0.,  0.],
        [-1.,  1., -1., -1.,  0.],
        [-1.,  0.,  0.,  1.,  1.],
        [-1.,  0.,  1., -1.,  0.]])

# Model and inference

In [22]:
def model(freq_matrix):
  '''A Pyro model for the TIRM model.

  Arguments:
    freq_matrix: The frequency matrix of observations.

  Notes:
    Currently not sampling alpha.
  '''
  # Model parameters
  theta = pyro.param("theta", torch.tensor(0.5), constraint=constraints.interval(0., 1.0))
  pi = pyro.param("pi", torch.tensor(0.5), constraint=constraints.interval(0., 1.0))
  alpha = pyro.param("alpha", torch.tensor(10), constraint=constraints.positive)
  # Useful variables
  num_objs = freq_matrix.size()[0]
  num_preds = freq_matrix.size()[1]
  obj_axis = pyro.plate("obj_axis", num_objs)
  pred_axis = pyro.plate("pred_axis", num_preds)
  # p(Z), modeled by a CRP
  freqs = [] # number of customers at each table
  for i in range(num_objs):
    probs = torch.tensor(freqs + [alpha])
    probs /= probs.sum()
    z_sample = pyro.sample(f"z_{i}", dist.Categorical(probs))
    z_item = z_sample.item()
    if z_item >= len(freqs):
      freqs += [1.]
    else:
      freqs[z_item] += 1.
  # We sample z ~ p(alpha)
  with obj_axis:
    z = pyro.sample("z", dist.Categorical(probs))
  type_axis = pyro.plate("type_axis", max(z)+1)
  # p(R | Z)
  with obj_axis, type_axis:
    r = pyro.sample("r", dist.Bernoulli(theta))
  # Pad R since it changes depending on z
  r = F.pad(input=r, pad=(0,0,0,num_objs-r.shape[0]), mode='constant', value=0)
  with obj_axis, pred_axis:
    # p (T | R, Z)
    t = pyro.sample("t", dist.Bernoulli(r[z]*pi))
    # p (D | T, R, Z)
    obs = t+(r[z]-1)
    d = pyro.sample("d", dist.Delta(obs), obs=freq_matrix)
  return z, r, t, d

In [20]:
def guide(freq_matrix):
  # Model parameters
  theta = pyro.param("theta", torch.tensor(.7), constraint=constraints.interval(0., 1.0))
  # pi = pyro.param("pi", torch.tensor(.7), constraint=constraints.interval(0., 1.0))
  alpha = pyro.param("alpha", torch.tensor(10), constraint=constraints.positive)
  # Useful variables
  num_objs = freq_matrix.size()[0]
  num_preds = freq_matrix.size()[1]
  freqs = [] # number of customers at each table
  for i in range(num_objs):
    probs = torch.tensor(freqs + [alpha])
    probs /= probs.sum()
    z_sample = pyro.sample(f"z_{i}", dist.Categorical(probs))
    z_item = z_sample.item()
    if z_item >= len(freqs):
      freqs += [1.]
    else:
      freqs[z_item] += 1.
  obj_axis = pyro.plate("obj_axis", num_objs)
  # pred_axis = pyro.plate("pred_axis", num_preds)
  with obj_axis:
    z = pyro.sample("z", dist.Categorical(probs))
  type_axis = pyro.plate("type_axis", max(z)+1)
  with obj_axis, type_axis:
    r = pyro.sample("r", dist.Bernoulli(theta))
  r = F.pad(input=r, pad=(0,0,0,num_objs-r.shape[0]), mode='constant', value=0)
  with obj_axis, pred_axis:
    t = pyro.sample("t", dist.Bernoulli(r[z]*pi))
    # obs = t+(r[z]-1)
    # d = pyro.sample("d", dist.Delta(obs))

## Variational inference

In [1007]:
pyro.clear_param_store()
svi = SVI(model, guide, pyro.optim.Adam({"lr": 1}), loss=pyro.infer.TraceEnum_ELBO())
num_steps = 1000
for step in range(num_steps):
    loss = svi.step(r)
    if step % 100 == 0:
        print("Step {}: loss = {:.2f}".format(step, loss))

2 tensor(2)


ValueError: Error while computing log_prob at site 'r':
Value is not broadcastable with batch_shape+event_shape: torch.Size([2, 5]) vs torch.Size([2, 2]).
Trace Shapes:      
 Param Sites:      
        theta      
        alpha      
Sample Sites:      
     z_0 dist     |
        value     |
     log_prob     |
     z_1 dist     |
        value     |
     log_prob     |
       z dist   2 |
        value   2 |
     log_prob   2 |
       r dist 2 2 |
        value 2 5 |

In [976]:
posterior_samples = {param: pyro.param(param).detach().clone() for param in pyro.get_param_store().keys()}

In [977]:
posterior_samples

{'theta': tensor(1.0000), 'pi': tensor(0.0800), 'alpha': tensor(10.)}

## MCMC

In [955]:
pyro.clear_param_store()
nuts_kernel = NUTS(model=model)
mcmc = MCMC(kernel=nuts_kernel, num_samples=1000, num_chains=10, warmup_steps=100)
posterior = mcmc.run(d)

Sample [0]: 100%|█████████████████████████████████| 1100/1100 [00:00, 30432.20it/s, step size=1.00e+00, acc. prob=1.000]                                                                                           


tensor([1.])
tensor([0.0909, 0.9091])
tensor([0.0833, 0.0833, 0.8333])
tensor([0.0769, 0.0769, 0.0769, 0.7692])
tensor([0.0714, 0.0714, 0.0714, 0.0714, 0.7143])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])


Sample [1]: 100%|█████████████████████████████████| 1100/1100 [00:00, 37270.66it/s, step size=1.00e+00, acc. prob=1.000]                                                                                           


tensor([1.])
tensor([0.0909, 0.9091])
tensor([0.0833, 0.0833, 0.8333])
tensor([0.0769, 0.0769, 0.0769, 0.7692])
tensor([0.0714, 0.0714, 0.0714, 0.0714, 0.7143])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])


Sample [2]: 100%|█████████████████████████████████| 1100/1100 [00:00, 36940.31it/s, step size=1.00e+00, acc. prob=1.000]                                                                                           


...

Warmup:   0%|                                                                                                                                                                               | 0/1100 [00:00, ?it/s]

tensor([1.])
tensor([0.0909, 0.9091])
tensor([0.0833, 0.0833, 0.8333])
tensor([0.0769, 0.0769, 0.0769, 0.7692])
tensor([0.0714, 0.0714, 0.0714, 0.0714, 0.7143])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])


Warmup:   0%|                                                                                                                                                                               | 0/1100 [00:00, ?it/s]

tensor([1.])
tensor([0.0909, 0.9091])
tensor([0.1667, 0.8333])
tensor([0.1538, 0.0769, 0.7692])
tensor([0.1429, 0.1429, 0.7143])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])
tensor([1.])
tensor([0.0909, 0.9091])
Z_1
Sample: tensor([[[[0]]],


        [[[1]]]])
Sample Shape: torch.Size([2, 1, 1, 1])
Probs tensor([0.0909, 0.9091])


Sample [9]: 100%|█████████████████████████████████| 1100/1100 [00:00, 37103.41it/s, step size=1.00e+00, acc. prob=1.000]                                                                                           
